# TUTORIAL
#### Hi! First time here? You are in the right place to learn how to use the code 

In [1]:
using GW

In [2]:
# these two packages are needed only for the tutorial
using BenchmarkTools
using Base.Threads
nthreads()

8

In [3]:
# if the previous cell returns 1, you need to change your .bashrc file to include the following line:
# export JULIA_NUM_THREADS = the_number_of_threads_of_your_cpu (e.g.  JULIA_NUM_THREADS = 8) then restart the terminal (or use the command source ~/.bashrc)

## FIRST PART: the basics

#### In this first part we will calculate the SNR, the Fisher Matrix and invert it, finally we will obtain the errors estimates.
#### But first we need to decide a waveform model and a detector configuration

In [4]:
# check the waveforms available with this function.
# In the code the functions are indicated with the name of the module in which they are contained, a dot and the name of the function
# There are 4 modules: waveform, detector, catalog and UtilsAndConstants (called with uc)
_available_waveforms()
# TaylorF2 is a very basic waveform, not useful
# PhenomD is a good compromise between accuracy and speed for this tutorial. Valid only for BBH
# PhenomHM is the most accurate waveform, but it is also the slowest. The recommended one for the final results, valid only for BBH
# PhenomD_NRTidal is the PhenomD waveform with the addition of the tidal effects. Valid only for BNS
# PhenomNSBH is the waveform valid for NSBH

5-element Vector{String}:
 "TaylorF2"
 "PhenomD"
 "PhenomHM"
 "PhenomD_NRTidal"
 "PhenomNSBH"

##### The main functions are indicated with the capital letter.
##### The other functions are auxiliary functions, indicated with the underscore.

In [5]:
# check the detectors available with this function
_available_detectors()
# the length in km is the arm length of the detector and it is not needed to call the detector
# e.g. _available_detectors("CE1Id")

11-element Vector{String}:
 "CE1Id, 40km"
 "CE2NM, 20km"
 "CE2NSW, 20km"
 "ETS, 10km"
 "ETLS, 10km"
 "ETMR, 10km"
 "ETLMR, 10km"
 "LIGO_L"
 "LIGO_H"
 "VIRGO"
 "KAGRA"

In [6]:
# now we can decide the waveform, remember the parenthesis (they are needed to define the model structure)

wfPhenomD = PhenomD()
# or use the function _available_waveforms()
wfPhenomD = _available_waveforms("PhenomD")

# then decide the detector
CE_1= CE1Id
# or use the function _available_detectors()
CE_1 = _available_detectors("CE1Id");

# to create a network of detectors put them in an array

CE_2 = CE2NM
ET = ETS

network = [CE_1, CE_2, ET]

3-element Vector{Detector}:
 Detector(0.7649254512715546, -1.9691677285626024, -0.7853981633974483, 1.5707963267948966, 'L', [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622  …  4698.324157477345, 4730.924097361, 4763.750236213645, 4796.804143546573, 4830.087399761323, 4863.601596225268, 4897.348335347697, 4931.32923065641, 4965.5459068749, 4999.999999999999], [5.320053993725279e-47, 4.977802675091629e-47, 4.663048207778976e-47, 4.373069475460877e-47, 4.1054566210234874e-47, 3.8580723586682724e-47, 3.6290183152679144e-47, 3.416605735864509e-47, 3.219329997795618e-47, 3.035848361932253e-47  …  1.1542927889652714e-48, 1.1798913424045747e-48, 1.206559811213688e-48, 1.2342718680182324e-48, 1.2630057938779226e-48, 1.2927441289587634e-48, 1.3234733704060701e-48, 1.3551836909058734e-48, 1.387868689337051e-48, 1.421525176464475e-48], "CE1Id")
 Detector(0.

In [7]:
# Let's calculate the SNR of an event with a given waveform and detector

# first we need to define the parameters of the event
mc = 30.0 # chirp mass at the detector frame in solar masses
η = 0.2 # symmetric mass ratio, dimensionless 
χ_1 = 0.1 # dimensionless spin of the most massive BH
χ_2 = 0.2 # dimensionless spin of the least massive BH
dL = 1.0 # luminosity distance in Gpc
θ = 45. * pi/180 # inclination angle in radians
ϕ = 45. * pi/180 # polarization angle in radians
ι = 45. * pi/180 # inclination angle in radians
ψ = 45. * pi/180 # polarisation angle in radians
tcoal = 0.0 # time of coalescence in fraction of a day, [0,1]
Φ_coal = 0.0; # phase of coalescence in radians

parameters = [mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal];

In [8]:
snr = SNR(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network)
# the function returns the SNR of the event, the order of parameters is very important in the code
# first there is the waveform, then there are the intrinsic parameters of the event, then the extrinsic parameters and finally the detector
# in particular, after the waveform, start with the chirp mass, then the symmetric mass ratio, then the spins, then the luminosity distance,
# then the angles, then the time of coalescence and finally the phase of coalescence

# the code requires the least amount of information, so in the SNR there is no need for the phase to coalescence,
# since the SNR depends only on the amplitude, which is independent of the phase.
# Thus the function SNR gives an error if you put also the phase of coalescence

777.6094582520611

In [9]:
# check that you obtained the correct result
snr == 777.6094582520611    # true

true

In [10]:
#check with the snr calculated by another code (GWFAST)
snrGWFAST = 777.60945825
isapprox(snr,snrGWFAST, rtol = 1e-10)    # true with relative tolerance 1e-10

true

In [11]:
# We can now move to the Fisher matrix of that event but choose a network of detectors
myfisher = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, coordinate_shift = false)
# It should take a minute or so because the code needs to be compiled first, each subsequent call will be blitzing fast
# to simplify the notation you can also write 
#myfisher = FisherMatrix(wfPhenomD, parameters..., network)

# the option coordinate_shift is related to T detectors (ET in this case) and represents the fact that the three detectors inside ET are not in the exact same position.
# This produces a difference in phase that improves the estimation of the parameters, but this option is not implemented in GWFAST, so we put it false to do a comparison.
# The default is true, so if you want to use it, you can just remove the option.

# the fisher is a 11x11 matrix (in the BBH case), the order is the same as the inputs.

# the units of the fisher matrix are the inverse of the square of the units of the parameters
# in particular the units of the chirp mass are solar masses, the units of the symmetric mass ratio are dimensionless, the units of the spins are dimensionless,
# the units of the luminosity distance are Gpc, the units of the angles are radians, the units of the time of coalescence are fraction of a day and the units of the phase of coalescence are radians.
# The errors keep the same units except for the time of coalescence, which is converted to seconds.

11×11 Matrix{Float64}:
      1.25029e10  -4.95201e10      -1.29424e10  …  -2.00353e10      8.67874e7
     -4.95201e10   2.02027e11       5.24711e10      9.19228e10     -3.40806e8
     -1.29424e10   5.24711e10       1.36461e10      2.32925e10     -8.92274e7
     -3.61225e9    1.44675e10       3.77269e9       5.85595e9      -2.4962e7
 -15025.3         -6.28205e5   -70159.6            -0.163545        0.0
     -1.44548e8    6.57912e8        1.67127e8   …   4.32171e8      -9.64147e5
     -2.43649e8    1.03555e9        2.66514e8       6.58424e8      -1.66962e6
      4.2846e6    -1.82023e7       -4.64556e6      -6.23763e6   29225.0
      1.74589e8   -6.85794e8       -1.79541e8      -2.71056e8       1.21621e6
     -2.00353e10   9.19228e10       2.32925e10      7.69501e10     -1.34815e8
      8.67874e7   -3.40806e8       -8.92274e7   …  -1.34815e8       6.04676e5

In [12]:
# this is a comparison of the Fisher matrix calculated by the code and the Fisher matrix calculated by GWFAST, we only compare one row but the result is valid for the whole matrix (in the limit of machine precision)
fisher_first_row_GWFAST = [1.25028777e+10, -4.95201244e+10, -1.29423577e+10, -3.61224785e+09, -1.50253200e+04, -1.44547812e+08, -2.43648573e+08, 4.28460151e+06, 1.74588867e+08, -2.00352948e+10,
                        8.67874125e+07]
isapprox(myfisher[1,:], fisher_first_row_GWFAST, rtol = 1e-8)    # true with relative tolerance 1e-8


true

In [13]:
# we can then calculate the covariance matrix by inverting the Fisher matrix (uc refers to the utils and constants module)
mycovariance = CovMatrix(myfisher)

11×11 Matrix{Float64}:
  7.23242e-7  -3.2975e-7    1.08466e-6   …   3.0717e-8   -3.32002e-5
 -3.2975e-7    6.54296e-7  -1.1187e-5        4.10002e-7   1.26942e-5
  1.08466e-6  -1.1187e-5    0.000253739     -1.02455e-5  -0.00012387
  2.16537e-6   2.7996e-5   -0.000698006      2.89052e-5   0.000192447
 -1.01296e-7   5.55902e-7  -1.14271e-5       4.84782e-7  -9.02915e-6
 -7.56714e-9   6.4974e-9   -1.96666e-7   …   5.14175e-9  -3.3578e-7
 -1.01271e-8   2.18286e-8  -2.66614e-7      -1.61353e-8   8.01447e-6
 -2.95392e-8  -2.48429e-7   5.97057e-6      -2.87899e-7   5.39887e-6
 -1.00872e-7  -7.77319e-8   2.4162e-6       -9.98621e-8  -0.000267982
  3.0717e-8    4.10002e-7  -1.02455e-5       4.24692e-7   2.94364e-6
 -3.32002e-5   1.26942e-5  -0.00012387   …   2.94364e-6   0.00280405

In [14]:
# we can now calculate the errors on the parameters
myerrors = Errors(mycovariance)
parameters_string = ["mc", "η", "χ_1", "χ_2", "dL", "θ", "ϕ", "ι", "ψ", "tcoal", "Φ_coal"]

for i in eachindex(myerrors)
    println("The error on $(parameters_string[i]) is $(myerrors[i])")
end

The error on mc is 0.0008504360749730822
The error on η is 0.0008088856061285265
The error on χ_1 is 0.01592917387229465
The error on χ_2 is 0.044367410029956963
The error on dL is 0.007751205023032693
The error on θ is 0.0004336067649812443
The error on ϕ is 0.0019172151420981257
The error on ι is 0.009008641014767917
The error on ψ is 0.011691407428797717
The error on tcoal is 0.0006516841768254732
The error on Φ_coal is 0.05295327234454468


In [15]:
# check the values of the errors with the values calculated by GWFAST
errors_GWFAST = [0.00085044, 0.00080889 , 0.01592917, 0.04436741, 0.00775121, 0.00043361, 0.00191722, 0.00900864, 0.01169141, 0.00065168, 0.05295327]
isapprox(myerrors, errors_GWFAST, rtol = 1e-6)    # true with relative tolerance 1e-6


true

In [16]:
# obtaint the sky area of the event
sky_area = SkyArea(mycovariance, θ) # the angle is in grad and represents the 90% credible interval
# for different credible intervals you can use the function uc.SkyArea(mycovariance, θ, credible_interval = 0.68) or 0.95

0.024299733801812353

### This is the end of the first part of the tutorial
#### In the second part we will dig a bit deeper into the code and see how to use the catalog module to generate a catalog of events

In [17]:
#################################################################################################################################################################################

# Second part: generate a catalog, the detector structure and loops
### The detector structure

In [18]:
# In the next cells we will go through the detector structure in the code,
# if you will use the detectors already present in the code you can skip this part.
# A detector in the code is a structure with the following fields

fieldnames(typeof(CE_1))


(:latitude_rad, :longitude_rad, :orientation_rad, :arm_aperture_rad, :shape, :fNoise, :psd, :label)

In [19]:
# to see what each field contains, use the following loop
for field in fieldnames(typeof(CE_1))
    println("$(field): $(getfield(CE_1, field))")
end

latitude_rad: 0.7649254512715546
longitude_rad: -1.9691677285626024
orientation_rad: -0.7853981633974483
arm_aperture_rad: 1.5707963267948966
shape: L
fNoise: [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622, 5.357966999113357, 5.395143957580921, 5.432578873262691, 5.4702735360287145, 5.508229748168284, 5.546449324476114, 5.584934092339116, 5.623685891823759, 5.66270657576406, 5.701998009850165, 5.7415620727175565, 5.7814006560368805, 5.821515664604383, 5.861909016432995, 5.902582642844027, 5.9435384885595175, 5.984778511795218, 6.026304684354212, 6.068118991721205, 6.110223433157443, 6.1526200217963085, 6.195310784739582, 6.238297763154348, 6.281583012370604, 6.325168601979518, 6.369056615932394, 6.4132491526403035, 6.457748325074419, 6.5025562608670455, 6.547675102413337, 6.593107006973743, 6.63885414677715, 6.684918709124732, 6.731302896494551,

In [20]:
# If you want to define your detector from scratch you can do it like

my_detector = Detector(43.827 * pi/180,
                                 -112.825 * pi/180,
                                  -45 * pi/180,
                                   90. * pi/180,
                                    'L',
                                 _readASD("useful_files/psds/ce_strain/cosmic_explorer.txt")...,
                                  "mydetector"
                                ) # in julia "" and '' are NOT equivalent, "s" is a string, 's' is a character

# the function _readASD reads the asd (amplitude spectral density) and outputs the psd (power spectral density) 
# It reads from a file which must have at least two columns, the first is the frequency and the second the asd
# if the file has more columns, to read other columns you can add them as arguments of the function _readASD("path", col=[1,3]) for example to read the first and third column
# the function returns the frequency and the psd, so you can use the splat operator (the dots) to pass the arguments to the Detector structure

# the detector structure is mutable, so you can change the values of the fields as you like, but remember that the fields are in the order of the constructor and you need to initialize all of them


Detector(0.7649254512715546, -1.9691677285626024, -0.7853981633974483, 1.5707963267948966, 'L', [5.000000000000001, 5.034693157380139, 5.069627037794076, 5.104803311530235, 5.140223660466547, 5.175889778150882, 5.211803369882009, 5.247966152791138, 5.284379855924022, 5.321046220323622  …  4698.324157477345, 4730.924097361, 4763.750236213645, 4796.804143546573, 4830.087399761323, 4863.601596225268, 4897.348335347697, 4931.32923065641, 4965.5459068749, 4999.999999999999], [5.320053993725279e-47, 4.977802675091629e-47, 4.663048207778976e-47, 4.373069475460877e-47, 4.1054566210234874e-47, 3.8580723586682724e-47, 3.6290183152679144e-47, 3.416605735864509e-47, 3.219329997795618e-47, 3.035848361932253e-47  …  1.1542927889652714e-48, 1.1798913424045747e-48, 1.206559811213688e-48, 1.2342718680182324e-48, 1.2630057938779226e-48, 1.2927441289587634e-48, 1.3234733704060701e-48, 1.3551836909058734e-48, 1.387868689337051e-48, 1.421525176464475e-48], "mydetector")

### Catalogs

In [21]:
# To create a catalog you need to decide the  number of events and the population (BBH, BNS or NSBH)
# this function writes the catalog as a .h5 file in the folder catalogs/
# the function returns the parameters of the events in the catalog
nEvents = 1_000
name = "my_first_catalog.h5"
typeof_events = "BBH"
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal = GenerateCatalog(nEvents, typeof_events, name_catalog = name)


([34.71139394215437, 48.60013127525831, 21.821309349452864, 69.18064848089311, 33.08818388904705, 69.562828198075, 19.150372069916, 174.71553233739692, 30.122096796669183, 20.41837816846945  …  17.562439093497854, 71.570064534699, 19.110528656177618, 24.37383941194993, 29.333236876715638, 45.72043203910072, 19.24383766637213, 11.180046415225432, 69.72320006943622, 69.16435106145826], [0.2498299894360189, 0.1823638402401015, 0.24999945884851502, 0.2499673327558577, 0.24999999403166429, 0.24915181729463232, 0.24987973383649645, 0.24999274173075453, 0.24812165481072707, 0.24563812457858325  …  0.24760306670902066, 0.24945317934343172, 0.24804545067910141, 0.23568407824414564, 0.18799374149751166, 0.24841159118189335, 0.24997913285461235, 0.24918081427636415, 0.24933812757631418, 0.2499881357245982], [0.07039160122428045, 0.20706008957170052, 0.22988166732907703, 0.26083878139833816, 0.38113048625435564, 0.2008825283241756, 0.31794918521412785, 0.43479669003980165, 0.3374781192063861, 0.15

In [22]:
### After you create the catalog you can read it with the following function
# note that is also prints some useful information about the catalog contained in the .h5 file
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal = ReadCatalog(name)

Attributes: ["SFR", "format", "number_events", "population", "seed", "time_delay_in_Myrs", "total_number_sources_yr"]
SFR: Madau&Dickinson
format: GWJulia
number_events: 1000
population: BBH
seed: 5006
time_delay_in_Myrs: 10.0
total_number_sources_yr: 35787.84461570478
Parameters: ["Lambda1", "Lambda2", "chi1", "chi2", "dL", "eta", "iota", "mc", "phi", "phiCoal", "psi", "tcoal", "theta", "z"]


([34.71139394215437, 48.60013127525831, 21.821309349452864, 69.18064848089311, 33.08818388904705, 69.562828198075, 19.150372069916, 174.71553233739692, 30.122096796669183, 20.41837816846945  …  17.562439093497854, 71.570064534699, 19.110528656177618, 24.37383941194993, 29.333236876715638, 45.72043203910072, 19.24383766637213, 11.180046415225432, 69.72320006943622, 69.16435106145826], [0.2498299894360189, 0.1823638402401015, 0.24999945884851502, 0.2499673327558577, 0.24999999403166429, 0.24915181729463232, 0.24987973383649645, 0.24999274173075453, 0.24812165481072707, 0.24563812457858325  …  0.24760306670902066, 0.24945317934343172, 0.24804545067910141, 0.23568407824414564, 0.18799374149751166, 0.24841159118189335, 0.24997913285461235, 0.24918081427636415, 0.24933812757631418, 0.2499881357245982], [0.07039160122428045, 0.20706008957170052, 0.22988166732907703, 0.26083878139833816, 0.38113048625435564, 0.2008825283241756, 0.31794918521412785, 0.43479669003980165, 0.3374781192063861, 0.15

### SNRs with for loops and multi-threading

In [23]:
# calculate the SNR for 100 events

snrs = Vector{Float64}(undef, nEvents) # create a vector to store the snrs

@btime for ii in 1:nEvents  # SNR of the first 100 events, the @btime macro is used to measure the time of the operation
    snrs[ii]=SNR(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], network)
end


  2.285 s (9072979 allocations: 1.20 GiB)


In [24]:
# if the number of events is larger than 100 we suggest to go multi-thread (up to now everything was done with a single thread)
# check the number of threads available with
nthreads()


8

In [25]:
# calculate the SNR, this time with multi-threading 

snrs = Vector{Float64}(undef, nEvents)
@btime @threads for ii in 1:nEvents # the @threads macro is used to parallelize the for loop
    snrs[ii]=SNR(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], network)
end

# the time should be smaller than the previous one, if not, try to increase the number of sources

  567.413 ms (9076425 allocations: 1.20 GiB)


In [26]:
# All that we did so far can be automatized with the SNR function
# we just need to pass all the parameter to the SNR function, it will already parallelize the calculation
# it will return the SNRs and if you use the flag "auto_save=true" it will save them in a file called SNRs.h5 in the folder output/BBH
# to change the destination folder you can use the flag "name_folder = "path/to/folder""

snrs = SNR(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network, auto_save=true);
# the output is a vector with the SNRs of the events

Computing SNRs... 100%|██████████████████████████████████| Time: 0:00:01


SNRs computed!
The evaluation took: 1.261683389 seconds.


1000-element Vector{Float64}:
  25.13906832480633
 161.27718487668164
  11.997850128467615
 105.62430583656956
  31.487793178387204
  51.41508098632884
  35.2936815883746
  70.07130088094563
  24.801754988350513
  36.67756272836259
   ⋮
 113.70843149049749
  18.682004879244705
  83.31487973359478
 126.07794777379581
  90.06426212094252
  14.39583731021668
 112.67214715596117
 235.3968615623581
  28.638420879375232

### Fisher matrix with for loops and multi-threading

In [27]:
# We can reproduce the steps we did before for the SNRs

nPar = 11 # number of parameters of the binary system for BBH
# to speed things up we do not calculate the Fisher matrix for all the events, but only for the first 100
Fisher = Array{Float64}(undef, 100, nPar, nPar)   # create a 3D array to store the Fisher matrices

@time for ii in 1:100
    Fisher[ii,:,:] = FisherMatrix(wfPhenomD, mc[ii], η[ii], χ_1[ii], χ_2[ii], dL[ii], θ[ii], ϕ[ii], ι[ii], ψ[ii], tcoal[ii], Φ_coal[ii], network)
end

# here we use the macro @time to measure the time of the operation
# the time takes also into account the compilation time, so the first time you run the code it will be longer than the subsequent times
# instread @btime does more runs of the code to give a more accurate estimate of the time

  6.573206 seconds (22.08 M allocations: 5.946 GiB, 5.05% gc time, 12.22% compilation time)


In [28]:
# As before we can go multi-threading with the macro @threads

Fisher = Array{Float64}(undef, 100, nPar, nPar)
@time @threads for i in 1:100
    Fisher[i,:,:] = FisherMatrix(wfPhenomD, mc[i], η[i], χ_1[i], χ_2[i], dL[i], θ[i], ϕ[i], ι[i], ψ[i], tcoal[i], Φ_coal[i], network)
end


 10.865927 seconds (37.59 M allocations: 6.899 GiB, 8.13% gc time, 643.06% compilation time)


In [29]:
# To do everything together we can use the function FisherMatrix passing directly the parameters of the events
# the function will calculate the Fisher matrix for all the events and save them in a file called fisher.h5 in the folder output/BBH
# to change the destination folder you can use the flag "folder = "path/to/folder""
# the function will also return the Fisher matrices
# the function calculates also the SNRs, so you do not need to calculate them before
# if you want to return the SNRs use the flag "return_SNR = true"
# if you save the Fisher matrices you also save the SNRs if the return_SNR flag is true 

Fisher, SNRs = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, return_SNR = true, auto_save=true);

# the output of the FisherMatrix function is a 3D array with the Fisher matrices of the events and has the same structure as the Fisher array created
# before. Thus Fisher[1,:,:] is the Fisher matrix of the first event, Fisher[:,1,1] is the first element of the Fisher matrix of all the events and so on.

Computing Fishers and SNRs... 100%|██████████████████████| Time: 0:00:15


Fisher matrices and SNRs computed!
The evaluation took: 15.630975983 seconds.


([6.276362234013022e6 -1.832568484050357e8 … -1.6137831374691725e7 62915.589345271306; 4.3033069574350126e7 -5.280448764179992e8 … -2.1406174935946697e8 1.0575320933761622e6; … ; 1.381857921328789e7 -7.282328671784648e8 … -2.0225080059067366e8 874566.9378786708; 201023.63249659713 -7.955577308098038e7 … -2.8547025751771843e6 12818.1495338067;;; -1.832568484050357e8 5.49428192108347e9 … 5.488386457362261e8 -1.8280382196540332e6; -5.280448764179992e8 6.577315488054155e9 … 2.856371322506457e9 -1.2937536649370767e7; … ; -7.282328671784648e8 3.993776633711746e10 … 1.2326366053822989e10 -4.58840476870416e7; -7.955577308098038e7 3.3568825585095245e10 … 1.3907520485207257e9 -5.014991900922793e6;;; -4.846004636511993e6 1.438017929858269e8 … 1.351416757717213e7 -48399.26857436176; -1.3931670323646218e8 1.7289955778515997e9 … 7.374329004105092e8 -3.4156558889461122e6; … ; -4.49612849741042e7 2.4131556182147923e9 … 7.015081505772412e8 -2.8380971529968367e6; -588083.9867994685 2.3974512732321063e8 

In [30]:
# to add the correction due to Earth motion just add the flag "useEarthMotion = true"

Fisher, SNRs = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, return_SNR = true, auto_save=true, useEarthMotion = true);


CompositeException: TaskFailedException

    nested task error: MethodError: no method matching _deltLoc(::Float64, ::Float64, ::Vector{Float64}, ::DetectorCoordinates)
    
    Closest candidates are:
      _deltLoc(::Union{Float64, ForwardDiff.Dual}, ::Union{Float64, ForwardDiff.Dual}, !Matched::Union{Float64, ForwardDiff.Dual}, ::DetectorStructure; REarth_km, clight_kms)
       @ GW ~/GW.jl/src/detector.jl:260
    
    Stacktrace:
     [1] AmplitudeDet(model::PhenomD, f::Vector{Float64}, mc::Float64, eta::Float64, chi1::Float64, chi2::Float64, dL::Float64, theta::Float64, phi::Float64, iota::Float64, psi::Float64, tcoal::Float64, DetectorCoordinates::DetectorCoordinates, Lambda1::Float64, Lambda2::Float64; alpha::Float64, useEarthMotion::Bool, ampl_precomputation::Vector{Float64})
       @ GW.detector ~/GW.jl/src/detector.jl:458
     [2] SNR(model::PhenomD, mc::Float64, eta::Float64, chi1::Float64, chi2::Float64, dL::Float64, theta::Float64, phi::Float64, iota::Float64, psi::Float64

##### Changing the SNR threshold

In [31]:
# the SNRs are calculated before the Fisher because under a threshold the Fisher matrix is not calculated, so the SNRs are needed to filter the events.
# The threshold is set to 12, but you can change it with the flag "rho_thres = value", e.g. rho_thres = 10.
# if you want to calculate the Fisher matrix for all the events, you can set the threshold to nothing, "rho_thres = nothing",
# this way it will not calculate the SNRs

Fisher_no_thres = FisherMatrix(wfPhenomD, mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, network, rho_thres = nothing);


Computing Fishers... 100%|███████████████████████████████| Time: 0:00:15


Fisher matrices computed!
The evaluation took: 15.513418034 seconds.


In [32]:
# if an event is under the threshold, the Fisher matrix is not calculated and the code returns a matrix of zeros
# this choice was made to keep track of which event correspond to which Fisher matrix.
# To select only the events above the threshold just type

Fisher_above_thres = Fisher[Fisher[:,1,1].!=0,:,:];
println("The number of events above the threshold is $(size(Fisher_above_thres)[1])")


The number of events above the threshold is 979


In [33]:
# To read the Fisher matrices and the SNRs you can use the following function
Fishers, SNRs = _read_Fishers_SNRs("output/BBH/Fishers_SNRs.h5")

Attributes: ["Detectors", "What_this_file_contains", "number_events"]
Detectors: ["CE1Id", "CE2NM", "ETS"]
What_this_file_contains: This file contains the Fishers and the SNRs for 1000 events obtained with the PhenomD waveform model. The calculations are performed with the CE1Id,CE2NM,ETS detectors and the correction due to Earth Motion was false.
number_events: 1000
Keys: ["Fishers", "SNRs"]


([6.276362234013022e6 -1.832568484050357e8 … -1.6137831374691725e7 62915.589345271306; 4.3033069574350126e7 -5.280448764179992e8 … -2.1406174935946697e8 1.0575320933761622e6; … ; 1.381857921328789e7 -7.282328671784648e8 … -2.0225080059067366e8 874566.9378786708; 201023.63249659713 -7.955577308098038e7 … -2.8547025751771843e6 12818.1495338067;;; -1.832568484050357e8 5.49428192108347e9 … 5.488386457362261e8 -1.8280382196540332e6; -5.280448764179992e8 6.577315488054155e9 … 2.856371322506457e9 -1.2937536649370767e7; … ; -7.282328671784648e8 3.993776633711746e10 … 1.2326366053822989e10 -4.58840476870416e7; -7.955577308098038e7 3.3568825585095245e10 … 1.3907520485207257e9 -5.014991900922793e6;;; -4.846004636511993e6 1.438017929858269e8 … 1.351416757717213e7 -48399.26857436176; -1.3931670323646218e8 1.7289955778515997e9 … 7.374329004105092e8 -3.4156558889461122e6; … ; -4.49612849741042e7 2.4131556182147923e9 … 7.015081505772412e8 -2.8380971529968367e6; -588083.9867994685 2.3974512732321063e8 

In [34]:
# To extract the covariance do as before but with for loops
nPar = 11
covs = Array{Float64}(undef, nEvents, nPar, nPar)
for ii in 1:nEvents
    covs[ii,:,:] = CovMatrix(Fisher[ii,:,:])
end
# if the covariance matrix is singular, the function will return a matrix with zeros, this will happen if the event is poorly 
# resolved (usually in the case of one or two detectors). In this case, the code will also return a warning message, one message for each event.

In [35]:
# Then to extract the errors
errors = Array{Float64}(undef, nEvents, nPar)
for ii in 1:nEvents
    errors[ii,:] = Errors(covs[ii,:,:])
end

In [36]:
# To extract the sky area then
sky_areas = Vector{Float64}(undef, nEvents)
for i in 1:nEvents
    sky_areas[i] = SkyArea(covs[i,:,:], θ[i])
end

In [37]:
# through out the tutorial we inizialed the empty arrays with undef, but you can also initialize them with zeros
# e.g. covs = zeros(nEvents, nPar, nPar)

In [38]:
#################################################################################################################################################################################

## Third Part: BNS and NSBH

In [39]:
# We do as before but this time we need to include the deformabilities of the neutron stars
# Λ_1 and Λ_2, they are dimensionless, in the case of BBH they are set to 0.
# The function GenerateCatalog will generate the catalog with the deformabilities

nEvents = 1_000
name = "my_first_catalog_of_BNS.h5"
typeof_events = "BNS"
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, Λ_1, Λ_2 = GenerateCatalog(nEvents, typeof_events, name_catalog = name)

([6.0462481869402565, 4.947647021601716, 9.174850565354154, 10.17312433965025, 3.6753094721148885, 2.8044044684646794, 4.258201872090929, 7.441239707071147, 6.535106754062695, 3.811187022904197  …  3.3434904451475234, 4.189388789432076, 3.87081913091597, 7.3771435607208335, 6.442502511080437, 4.235774026615787, 6.49636796201784, 3.111443986371986, 4.228290323792195, 4.6024606471849125], [0.24999601648844083, 0.24191643105172275, 0.2497813930336011, 0.24916669259552468, 0.2440541629029104, 0.23394412480690008, 0.24977301852432535, 0.2445746054706368, 0.24990168918639089, 0.22915769436834563  …  0.2181561910013001, 0.24562260727136762, 0.24979351753306742, 0.2486112824186036, 0.249998973808445, 0.23136748562826887, 0.2498157419394441, 0.2428321622636334, 0.2436119252538299, 0.24948868843592653], [-0.0195996671535709, 0.03378999812973729, 0.02075054279204039, 0.023504244385281342, -0.04350657831183211, -0.03770344949942807, -0.002122234002775024, 0.04000497028972583, 0.04038386689011968, 

In [40]:
# we can read the catalog as before
mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, Λ_1, Λ_2 = ReadCatalog(name)

Attributes: ["SFR", "format", "number_events", "population", "seed", "time_delay_in_Myrs", "total_number_sources_yr"]
SFR: Madau&Dickinson
format: GWJulia
number_events: 1000
population: BNS
seed: 2072
time_delay_in_Myrs: 10.0
total_number_sources_yr: 222095.15335040318
Parameters: ["Lambda1", "Lambda2", "chi1", "chi2", "dL", "eta", "iota", "mc", "phi", "phiCoal", "psi", "tcoal", "theta", "z"]


([6.0462481869402565, 4.947647021601716, 9.174850565354154, 10.17312433965025, 3.6753094721148885, 2.8044044684646794, 4.258201872090929, 7.441239707071147, 6.535106754062695, 3.811187022904197  …  3.3434904451475234, 4.189388789432076, 3.87081913091597, 7.3771435607208335, 6.442502511080437, 4.235774026615787, 6.49636796201784, 3.111443986371986, 4.228290323792195, 4.6024606471849125], [0.24999601648844083, 0.24191643105172275, 0.2497813930336011, 0.24916669259552468, 0.2440541629029104, 0.23394412480690008, 0.24977301852432535, 0.2445746054706368, 0.24990168918639089, 0.22915769436834563  …  0.2181561910013001, 0.24562260727136762, 0.24979351753306742, 0.2486112824186036, 0.249998973808445, 0.23136748562826887, 0.2498157419394441, 0.2428321622636334, 0.2436119252538299, 0.24948868843592653], [-0.0195996671535709, 0.03378999812973729, 0.02075054279204039, 0.023504244385281342, -0.04350657831183211, -0.03770344949942807, -0.002122234002775024, 0.04000497028972583, 0.04038386689011968, 

In [41]:
# we can calculate the SNRs but we need to pay attention to the number and order of parameters
# the order is mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, detector, Λ_1, Λ_2

snrs = SNR(PhenomD_NRTidal(), mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, network, Λ_1, Λ_2, auto_save=true, name_folder = "BNS");

Computing SNRs... 100%|██████████████████████████████████| Time: 0:00:02


SNRs computed!
The evaluation took: 2.340538749 seconds.


In [42]:
# same as before we can calculate the Fisher matrices
# we also introduce another detector in the network
# the network is now [CE_1, CE_2, ET, my_detector]
my_network = [CE_1, CE_2, ET, my_detector]
Fisher_BNS, SNRs_BNS = FisherMatrix(PhenomD_NRTidal(), mc, η, χ_1, χ_2, dL, θ, ϕ, ι, ψ, tcoal, Φ_coal, my_network, Λ_1, Λ_2, return_SNR = true,
                                     auto_save=true, name_folder = "BNS/with_my_detector");
# note that I added to the name_folder "with_my_detector" to create a new folder

In [ ]:
mc[1], η[1], χ_1[1], χ_2[1], dL[1], θ[1], ϕ[1], ι[1], ψ[1], tcoal[1], Φ_coal[1], Λ_1[1], Λ_2[1]

In [ ]:
Ampl(PhenomD_NRTidal(),fgrid[end], mc[1], η[1], χ_1[1], χ_2[1], dL[1] , Λ_1[1], Λ_2[1])   

In [ ]:
using ForwardDiff
ForwardDiff.jacobian(x -> Ampl(PhenomD_NRTidal(),[10.,30.], x[1], x[2], χ_1[1], χ_2[1], dL[1] , Λ_1[1], Λ_2[1]), [mc[1], η[1]])

In [ ]:
ForwardDiff.jacobian(x -> Ampl(PhenomD_NRTidal(),fgrid[end-1:end], x...) , [mc[1], η[1], χ_1[1], χ_2[1], dL[1] , Λ_1[1], Λ_2[1]])

In [ ]:
idx = 7
model = PhenomD_NRTidal()
if isnothing(nothing)
    fcut = _fcut(model, mc[idx], η[idx], Λ_1[idx], Λ_2[idx])
else
    fcut_tmp = _fcut(model, mc, η, Λ_1, Λ_2)
    fcut = ifelse(fcut_tmp > fmax, fmax, fcut_tmp)
end


fgrid = 10 .^ (range(log10(2.), log10(fcut), length = 1000));

cc=ForwardDiff.jacobian(x -> Ampl(PhenomD_NRTidal(),fgrid, x...) , [mc[idx], η[idx], χ_1[idx], χ_2[idx], dL[idx] , Λ_1[idx], Λ_2[idx]])

In [ ]:
# find NaN in a matrix
findall(isnan, cc)
# put NaN to 0.
cc[isnan.(cc)] .= 0.0
cc[965:970,:]


In [ ]:
ForwardDiff.jacobian(x -> Phi(PhenomD_NRTidal(),fgrid, x...) , [mc[1], η[1], χ_1[1], χ_2[1] , Λ_1[1], Λ_2[1]])

In [ ]:
pars = [2.9094774313329057, 0.24923433537935963, -0.03592996118963426, 0.045423608232945445, 10.133603271038025, 1.4521963062898466, 5.45688227115126, 2.063833966625118, 0.05287346159004536, 0.08678679655837751, 0.10598111930132251, 869.5550395246836, 1432.3324633160732]


In [ ]:
SNRs_BNS

In [ ]:
Fisher_BNS[,:,:]

In [ ]:
findall(iszero, Fisher_BNS[1,:,:])

In [ ]:
# To read the Fisher matrices and the SNRs you can use the following function
Fishers, SNRs = _read_Fishers_SNRs("output/BNS/with_my_detector/Fishers_SNRs.h5")